In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')

In [71]:
test = pd.read_csv('test.csv')

In [51]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [4]:
len(train)

7613

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
id          3263 non-null int64
keyword     3237 non-null object
location    2158 non-null object
text        3263 non-null object
dtypes: int64(1), object(3)
memory usage: 102.0+ KB


In [8]:
train['location'].fillna('None',inplace = True)
train['keyword'].fillna('None',inplace = True)

In [72]:
test['location'].fillna('None',inplace = True)
test['keyword'].fillna('None',inplace = True)

In [10]:
train.head()

,id,keyword,location,text,target
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1
1,4,None,None,Forest fire near La Ronge Sask. Canada,1
2,5,None,None,All residents asked to 'shelter in place' are ...,1
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1


In [11]:
import spacy

In [13]:
nlp = spacy.load('en_core_web_sm')

# Better way to create vectors?

In [15]:
vectors = np.array([nlp(tweet.text).vector for idx, tweet in train.iterrows()])

In [17]:
from sklearn.model_selection import train_test_split

In [19]:
vectors

array([[ 0.13452284,  0.23726209, -0.11959665, ...,  1.0300049 ,
         2.0431578 ,  1.0130912 ],
       [ 1.0309796 , -1.241845  , -0.70834446, ...,  2.0340476 ,
         1.3641878 , -0.49217844],
       [-0.07072449,  1.0874252 , -1.0262684 , ...,  0.14652216,
         0.01183708,  0.8379208 ],
       ...,
       [ 0.81655306, -0.525268  ,  0.08102124, ...,  0.07882321,
         0.07872479,  1.1387758 ],
       [ 1.3380736 , -0.34899577,  0.02064458, ..., -0.07607778,
         0.99538714,  1.0481292 ],
       [ 0.8571995 , -0.5512727 ,  0.12522113, ..., -0.12230115,
         1.0029335 ,  0.71867335]], dtype=float32)

In [21]:
train2 = pd.DataFrame(vectors)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(vectors,train['target'],test_size = .3, random_state = 42)

In [24]:
from sklearn.svm import LinearSVC

In [27]:
svc = LinearSVC(random_state = 42, dual = False)

In [28]:
svc.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [29]:
from sklearn.metrics import accuracy_score, classification_report

In [30]:
svc_pred = svc.predict(X_test)

In [31]:
print(classification_report(y_test,svc_pred))

              precision    recall  f1-score   support

           0       0.72      0.79      0.75      1318
           1       0.66      0.58      0.62       966

   micro avg       0.70      0.70      0.70      2284
   macro avg       0.69      0.68      0.68      2284
weighted avg       0.69      0.70      0.69      2284



In [33]:
print(accuracy_score(y_test,svc_pred))

0.696584938704028


# Could be better.... maybe try the centering approach? Pipeline?

# pipeline approach

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),
                    ('clf',LinearSVC()),
])

In [41]:
X = train['text']
y = train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [42]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [43]:
predictions = text_clf.predict(X_test)

In [45]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82      1446
           1       0.77      0.74      0.75      1067

   micro avg       0.79      0.79      0.79      2513
   macro avg       0.79      0.79      0.79      2513
weighted avg       0.79      0.79      0.79      2513



In [48]:
print(accuracy_score(y_test,predictions))

0.7938718662952646


In [52]:
#create submission for easy pipeline

In [94]:
thing = text_clf.predict(test['text'])
submission = pd.DataFrame({
    'id': test.id,
    'target': thing
})
submission.to_csv('submission_pipeline_v1.csv', index = False)

# A bit better...centering approach? find some way to use locations/keywords?

scores .78425 on Kaggle leaderboard